# Set project environment
## Importing the packages

In [13]:
from gpt_tools.generate_answer import process_question

from flask import Flask, render_template, request, jsonify

import os
import openai
import uuid

import pandas as pd

from dotenv import load_dotenv

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
# load environmental variables
load_dotenv()

# directory definition
output_directory = os.environ.get("OUTPUT_DIRECTORY")
doc_source = os.environ.get("DOC_SOURCE_DIRECTORY")
persist_directory = os.environ.get('SIMPLEGPT_PERSIST_DIRECTORY')
embeddings_model_name = os.environ.get('EMBEDDINGS_MODEL_NAME')

used_model = "OPENAI"   
# used_model = ""

# Vectorstore and LLM-related functions

# App and flask-related functions

# Configure the flask app

In [15]:
app = Flask("brainGPT")


embeddings = OpenAIEmbeddings()


# loading the vectorstore
vectordb = Chroma(persist_directory=persist_directory,
                   embedding_function=embeddings)

@app.route("/")
def renderIndexPage():
    return render_template("index.html")


@app.route('/api/question', methods=['POST'])
def api_question():
    data = request.get_json()
    question = data.get('question')
    if question:
        answer = process_question(in_question=question,
                                  in_vectordb=vectordb)
        return jsonify(answer)
    else:
        return jsonify({"error": "Invalid question"}), 400
    

# Route to handle the form submission
@app.route('/submit', methods=['POST'])
def submit():
    question = request.form['question']  # Get the question from the submitted form
    answer = process_question(in_question=question,
                              in_vectordb=vectordb)  # Process the question to get the answer
#    answer = "Test answer"  # Process the question to get the answer
    return render_template('index.html', question=question, answer=answer)



# Main script

In [16]:
if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000)

 * Serving Flask app 'brainGPT'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Aug/2023 23:07:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2023 23:07:36] "GET /socket.io/?EIO=4&transport=polling&t=OdChKqQ HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2023 23:07:36] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2023 23:07:37] "GET /socket.io/?EIO=4&transport=polling&t=OdChL5h HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2023 23:07:40] "GET /socket.io/?EIO=4&transport=polling&t=OdChLj7 HTTP/1.1" 404 -
127.0.0.1 - - [06/Aug/2023 23:07:45] "GET /socket.io/?EIO=4&transport=polling&t=OdChMxS HTTP/1.1" 404 -
[2023-08-06 23:07:48,945] ERROR in app: Exception on /api/question [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/brainGPT/lib/python3.11/site-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/brainGPT/lib/python3.11/site-packages/flask/app.py", line 1486, in 